In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import re


# --- CHANGE IS HERE ---
# Set pandas display option to show full column content, preventing truncation.
pd.set_option('display.max_colwidth', None)

# Import our custom modules
from src import config, llm_client, planner


# ==============================================================================
# 1. Create the UI Widgets
# ==============================================================================

# --- Basic Setup Section ---
part_name_input = widgets.Text(value='Top Mounting Plate', description='Part Name:')
material_input = widgets.Dropdown(
    options=['Aluminum 6061-T6', 'Mild Steel', '42CrMo (Steel)', 'Titanium', 'Acetal (Delrin)'],
    value='Aluminum 6061-T6',
    description='Material:'
)

# --- Geometry & Features Description Section ---
prompt_text_template = """# Please describe your part in detail. 
# Providing the following information will lead to a better and more accurate plan.
"""
# --- Stock and Part Dimensions (CRITICAL for machine compatibility checks) ---
# Raw Stock Type: Block
# Raw Stock Size: L=110mm, W=60mm, H=20mm
# Final Part Bounding Box: L=100mm, W=50mm, H=15mm

# --- Key Machining Features ---
# - Four (4) M8 through-holes at each corner, 12mm from edges.
# - A central 40x20mm pocket, 8mm deep.
# - All sharp edges should have a 0.5mm chamfer.

description_input = widgets.Textarea(
    value=prompt_text_template,
    description='Geometry & Features:',
    layout={'height': '300px', 'width': '70%'},
    style={'description_width': 'initial'}
)

# --- Action Button and Output Area ---
generate_button = widgets.Button(description="Generate Process Plan", button_style='success')
output_area = widgets.Output()


# ==============================================================================
# 2. Define the Function to Run on Button Click (Full Logic)
# ==============================================================================

def on_generate_button_clicked(b):
    # Disable button to prevent multiple clicks
    generate_button.disabled = True
    
    with output_area:
        clear_output()
        print("Building description from inputs...")

        # A. Assemble the full description string from the UI components
        material = material_input.value
        full_description = f"""
        Part Name: {part_name_input.value}
        Material: {material}
        ---
        Geometry and Feature Description from user:
        {description_input.value}
        """
        
        print("Description built. Calling AI to generate the process plan...")

        try:
            # B. Run the full backend processing pipeline
            llm_response = llm_client.get_process_plan_from_llm(full_description)
            
            if not llm_response:
                clear_output()
                print("Error: Failed to get a valid response from the LLM. Please check the console for details.")
                return

            parsed_plan = planner.post_process_response(llm_response)
            
            # C. Try to extract dimension information and validate
            part_dimensions = planner.extract_dimensions_from_text(description_input.value)
            
            # Corrected variable name from 'part_dims' to 'part_dimensions'
            validated_plan = planner.validate_plan(parsed_plan, part_dimensions, material)
            
            # D. Display the final result clearly in the output area
            df = pd.DataFrame(validated_plan)
            
            clear_output()
            print("--- AI-Assisted CNC Process Plan ---")
            display(df)
        
        finally:
            # Re-enable button after the process is complete
            generate_button.disabled = False


generate_button.on_click(on_generate_button_clicked)


# ==============================================================================
# 3. Assemble and Display the Final UI
# ==============================================================================

ui_layout = widgets.VBox([
    widgets.HTML("<h3>1. Basic Setup</h3>"),
    part_name_input,
    material_input,
    widgets.HTML("<hr><h3>2. Part Geometry & Features Description</h3>"),
    description_input,
    widgets.HTML("<hr>"),
    generate_button,
    output_area
])

display(ui_layout)
